<a href="https://colab.research.google.com/github/Abby-Wale/CNN_For_Dementia/blob/main/ResNet_vs_CNN_Dementia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# deep-learning-project

Use the "Run" button to execute the code.

In [ ]:
!pip install opendatasets --upgrade --quiet
!pip install jovian --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import opendatasets as od
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
import math
import shutil
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
import jovian

In [ ]:
dataset_url = 'https://www.kaggle.com/datasets/sachinkumar413/alzheimer-mri-dataset/download?datasetVersionNumber=1'
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

In [ ]:
dementia_data = '/content/alzheimer-mri-dataset/Dataset'

In [ ]:
for cls in os.listdir(dementia_data):
    print(cls, ':', len(os.listdir(dementia_data + '/' + cls)))

In [ ]:
dataset = ImageFolder(dementia_data)
len(dataset)

In [ ]:
dataset[0]

In [ ]:
os.listdir(dementia_data)

"""the code below does the same as above"""

dataset.classes

In [ ]:
"""This code snippet involves retrieving an image and its 
corresponding label from a dataset object. The image and 
label are assigned to variables `img` and `label`. Then the
 retrieved image is displayed using `imshow()` function from the 
 `matplotlib.pyplot` library. The resulting image is displayed 
 in the Jupyter notebook output cell."""

img, label = dataset[120]
plt.imshow(img)

In [ ]:
"""This code initializes an `ImageFolder` object named `dataset` 
that contains a collection of images. The images are loaded from 
a directory `dementia_data`. A series of image transformations 
are applied to each image using `Compose` from 
`torchvision.transforms`. The transformations include resizing, 
random cropping, and converting the image to a tensor.
"""

dataset = ImageFolder(dementia_data, tt.Compose([tt.Resize(64),
                                                 tt.RandomCrop(64),
                                                 tt.ToTensor()]))

In [ ]:
img, label = dataset[120]
plt.imshow(img.permute((1, 2, 0)))

In [ ]:
"""This code calculates the size of the training, validation, 
and testing sets based on the total length of the dataset 
and the percentage split specified in `val_pct`. 
The calculated sizes of the training, validation, and 
testing sets are then printed."""
val_pct = 0.05
val_size = int(val_pct * len(dataset))
test_size = int(val_pct * len(dataset))
train_size = len(dataset) - (val_size + test_size)

train_size, val_size, test_size

In [ ]:
"""In this code snippet, `random_split()` function from PyTorch's `torch.utils.data` 
module is used to split the `dataset` into `train_ds`, `valid_ds` and `test_ds` 
based on the sizes specified by `train_size`, `val_size`, and `test_size` variables.
The lengths of `train_ds`, `valid_ds`, and `test_ds` are then displayed using the 
`len()` function."""

train_ds, valid_ds , test_ds = random_split(dataset, [train_size, val_size, test_size])
len(train_ds), len(valid_ds), len(test_ds)

In [ ]:
"""These lines of code are creating data loaders for the training, validation, and test datasets. 
A DataLoader is a PyTorch object that provides an iterable over a dataset, allowing us to access 
the data in batches. The `train_dl`, `valid_dl`, and `test_dl` are DataLoader objects created 
using the `train_ds`, `valid_ds`, and `test_ds` datasets, respectively. Each data loader has a 
batch size of 64, which means that during training, the model will receive input data in batches 
of 64 images at a time. The `shuffle` argument is set to True only for the training data loader, 
which means that the order of the images in each batch will be randomized for each epoch of training. 
The `num_workers` argument is set to 2, which means that two worker processes will be used to load 
the data in parallel. The `pin_memory` argument is set to True, which means that the data will be 
loaded into pinned memory, making it faster to transfer the data to the GPU."""

batch_size = 64

train_dl = DataLoader(train_ds, 
                      batch_size, 
                      shuffle=True, 
                      num_workers=2, 
                      pin_memory=True)

valid_dl = DataLoader(valid_ds, 
                    batch_size, 
                    num_workers=2, 
                    pin_memory=True)

test_dl = DataLoader(test_ds, 
                    batch_size, 
                    num_workers=2, 
                    pin_memory=True)

In [ ]:
"""The `show_batch` function takes a data loader object (`dl`) as input and displays a batch of 
images and their corresponding labels. It iterates over the data loader and retrieves a batch 
of images and their labels. It then uses the `make_grid` function from the `torchvision.utils` 
module to create a grid of the images. The `imshow` function from the `matplotlib.pyplot` module 
is used to display the grid of images in a figure with no x and y ticks. 
The `break` statement ensures that only one batch of images is displayed."""

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
        break
show_batch(train_dl)

##Defining Base Class



In [ ]:

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        "calculate loss for a batch of training data"
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        "calculate loss & accuracy for a batch of validation data"
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

##MODEL

In [ ]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

# ResNet9 Model
class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        # Input: 128 x 3 x 64 x 64
        self.conv1 = conv_block(in_channels, 64) # 128 x 64 x 64 x 64
        self.conv2 = conv_block(64, 128, pool=True) # 128 x 128 x 32 x 32
        self.res1 = nn.Sequential(conv_block(128, 128), # 128 x 128 x 32 x 32
                                  conv_block(128, 128)) # 128 x 128 x 32 x 32
        
        self.conv3 = conv_block(128, 256, pool=True) # 128 x 256 x 16 x 16
        self.conv4 = conv_block(256, 512, pool=True) # 128 x 512 x 8 x 8 
        self.res2 = nn.Sequential(conv_block(512, 512), # 128 x 512 x 8 x 8 
                                  conv_block(512, 512)) # 128 x 512 x 8 x 8 
        
        self.classifier = nn.Sequential(nn.AdaptiveMaxPool2d(1), # 128 x 512 x 1 x 1 
                                        nn.Flatten(), # 128 x 512
                                        nn.Dropout(0.2),
                                        nn.Linear(512, num_classes))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

# print
# class PrintShape(nn.Module):
#     def forward(self, x):
#         print(x.shape)
#         return x

# CNN Model
class DementiaCNNModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

            nn.Flatten(), 
            nn.Linear(256*8*8, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 4))
        
    def forward(self, xb):
        return self.network(xb)

In [ ]:
cnn_model = DementiaCNNModel()
cnn_model

In [ ]:
# cnn_model.conv2d[0].weight.device

In [ ]:
for images, labels in train_dl:
    print('images.shape:', images.shape)
    out = cnn_model(images)
    print('out.shape:', out.shape)
    print('out[0]:', out[0])
    break

In [ ]:
resnet_model = ResNet9(3, len(dataset.classes))
resnet_model

In [ ]:
resnet_model.conv1[0].weight.device

In [ ]:
torch.cuda.empty_cache()
for batch in train_dl:
    images, labels = batch
    print('images.shape', images.shape)
    print('images.device', images.device)
    preds = resnet_model(images)
    print('preds.shape', preds.shape)
    break

##Helper Functions Before Training The Model

In [ ]:
"""this block helps choose the available accelerator"""

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
torch.cuda.is_available()

In [ ]:
device = get_default_device()
device

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(valid_dl, device)
test_dl = DeviceDataLoader(test_dl, device)

##Training the Models


In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    """Evaluates the model's performance on the validation set"""
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
"""checking  models and evaluating results again before training"""

# evaluate cnn_model
cnn_model = to_device(DementiaCNNModel(), device)
cnn_result = evaluate(cnn_model, valid_dl)
print(f"CNN model evaluation: {cnn_result}")

# evaluate resnet_model
resnet_model = to_device(ResNet9(3, len(dataset.classes)), device)
resnet_result = evaluate(resnet_model, valid_dl)
print(f"ResNet model evaluation: {resnet_result}")


In [ ]:
num_epochs = 4
opt_func = torch.optim.Adam
lr = 0.0001

In [ ]:
print('CNN')
cnn_history = fit(num_epochs, lr, cnn_model, train_dl, valid_dl, opt_func)

print('\n')
print('ResNet')
resnet_history = fit(num_epochs, lr, resnet_model, train_dl, valid_dl, opt_func)

# for i in range(num_epochs):
#     print(f"Epoch {i+1}/{num_epochs}, CNN Loss: {cnn_history[i]['val_loss']:.5f}, CNN Acc: {cnn_history[i]['val_acc']:.5f}, ResNet Loss: {resnet_history[i]['val_loss']:.5f}, ResNet Acc: {resnet_history[i]['val_acc']:.5f}")



In [ ]:
print('CNN')
cnn_history = fit(num_epochs, lr, cnn_model, train_dl, valid_dl, opt_func)

print('\n')
print('ResNet')
resnet_history = fit(num_epochs, lr, resnet_model, train_dl, valid_dl, opt_func)
# for i in range(num_epochs):
#     print(f"Epoch {i+1}/{num_epochs}, \n CNN Loss: {cnn_history[i]['val_loss']:.5f}, CNN Acc: {cnn_history[i]['val_acc']:.5f},\n ResNet Loss: {resnet_history[i]['val_loss']:.5f}, ResNet Acc: {resnet_history[i]['val_acc']:.5f}")

In [ ]:
print('CNN')
cnn_history = fit(num_epochs, lr, cnn_model, train_dl, valid_dl, opt_func)

print('\n')
print('ResNet')
resnet_history = fit(num_epochs, lr, resnet_model, train_dl, valid_dl, opt_func)

# for i in range(num_epochs):
#     print(f"Epoch {i+1}/{num_epochs}, \n CNN Loss: {cnn_history[i]['val_loss']:.5f}, CNN Acc: {cnn_history[i]['val_acc']:.5f},\n ResNet Loss: {resnet_history[i]['val_loss']:.5f}, ResNet Acc: {resnet_history[i]['val_acc']:.5f}")

In [ ]:
print('CNN')
cnn_history = fit(num_epochs, lr, cnn_model, train_dl, valid_dl, opt_func)

print('\n')
print('ResNet')
resnet_history = fit(num_epochs, lr, resnet_model, train_dl, valid_dl, opt_func)
# for i in range(num_epochs):
#     print(f"Epoch {i+1}/{num_epochs}, \n CNN Loss: {cnn_history[i]['val_loss']:.5f}, CNN Acc: {cnn_history[i]['val_acc']:.5f},\n ResNet Loss: {resnet_history[i]['val_loss']:.5f}, ResNet Acc: {resnet_history[i]['val_acc']:.5f}")

In [ ]:
print('CNN')
cnn_history = fit(num_epochs, lr, cnn_model, train_dl, valid_dl, opt_func)

print('\n')
print('ResNet')
resnet_history = fit(num_epochs, lr, resnet_model, train_dl, valid_dl, opt_func)
# for i in range(num_epochs):
#     print(f"Epoch {i+1}/{num_epochs}, \n CNN Loss: {cnn_history[i]['val_loss']:.5f}, CNN Acc: {cnn_history[i]['val_acc']:.5f},\n ResNet Loss: {resnet_history[i]['val_loss']:.5f}, ResNet Acc: {resnet_history[i]['val_acc']:.5f}")



In [ ]:
print('CNN')
cnn_history = fit(num_epochs, lr, cnn_model, train_dl, valid_dl, opt_func)

print('\n')
print('ResNet')
resnet_history = fit(num_epochs, lr, resnet_model, train_dl, valid_dl, opt_func)
# for i in range(num_epochs):
#     print(f"Epoch {i+1}/{num_epochs}, \n CNN Loss: {cnn_history[i]['val_loss']:.5f}, CNN Acc: {cnn_history[i]['val_acc']:.5f},\n ResNet Loss: {resnet_history[i]['val_loss']:.5f}, ResNet Acc: {resnet_history[i]['val_acc']:.5f}")

In [ ]:
print('CNN')
cnn_history = fit(num_epochs, lr, cnn_model, train_dl, valid_dl, opt_func)

print('\n')
print('ResNet')
resnet_history = fit(num_epochs, lr, resnet_model, train_dl, valid_dl, opt_func)


In [ ]:
def plot_accuracies(history, ax, name):
    accuracies = [x['val_acc'] for x in history]
    ax.plot(accuracies, '-x')
    ax.set_xlabel('epoch')
    ax.set_ylabel('accuracy')
    ax.set_title(f'{name}: Acc vs epochs')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
plot_accuracies(cnn_history, ax1, 'CNN')
plot_accuracies(resnet_history, ax2, 'ResNet')
fig.tight_layout(pad=3.0)
plt.show()


# plot_accuracies(cnn_history), plot_accuracies(resnet_history)

In [ ]:
def plot_losses(history, ax):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    ax.plot(train_losses, '-bx')
    ax.plot(val_losses, '-rx')
    ax.set_xlabel('epoch')
    ax.set_ylabel('loss')
    ax.legend(['Training', 'Validation'])
    ax.set_title('Loss vs. No. of epochs')

fig, (ax1, ax2) = plt.subplots(1, 2)
plot_losses(cnn_history, ax1)
plot_losses(resnet_history, ax2)
plt.show()

In [ ]:
print("CNN last output: ", cnn_history[-1])
print("ResNet last output: ", resnet_history[-1])

In [ ]:
# jovian.reset()
# jovian.log_dataset(image_size=64)
# jovian.log_hyperparams(batch_size=128, 
#                        arch='ResNet9', 
#                        epochs=[5, 5, 5, 5], 
#                        lrs=[0.001, 0.001, 1e-4, 1e-4],
#                        opt=['Adam', 'Adam', 'Adam', 'SGD'])
# jovian.log_metrics(train_loss=history[-1]['train_loss'],
#                    val_acc=history[-1]['val_acc'],
#                    val_loss=history[-1]['val_loss'])

In [ ]:
def predict_image(img, model, classes):
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label
    return classes[preds[0].item()]

In [ ]:
def show_image_prediction(img, label, model):
    plt.imshow(img.permute((1, 2, 0)))
    pred = predict_image(img, model, dataset.classes)
    print('Target:', dataset.classes[label])
    print('Prediction:', pred)

In [ ]:
show_image_prediction(*test_ds[100], resnet_model)
show_image_prediction(*test_ds[100], cnn_model)

In [ ]:
show_image_prediction(*test_ds[10], resnet_model)
show_image_prediction(*test_ds[10], cnn_model)

In [ ]:
show_image_prediction(*test_ds[230], resnet_model)
show_image_prediction(*test_ds[230], cnn_model)

In [ ]:
show_image_prediction(*test_ds[5], resnet_model)
show_image_prediction(*test_ds[5], cnn_model)

In [ ]:
# torch.save(model.state_dict(), 'dementia-resnet9.pth')

###comments


Pooling and Flatten are different concepts while pooling you're reducing the number of elements and selecting a max, min, or average element from a set of multiple elements. On the other hand, flatten just changes the dimension of a list of elements from multiple dimensions to a single dimension, so if there are 12 elements in total after flatten you will also have 12 elements. 